<a href="https://colab.research.google.com/github/cristinapopescu19/Projects/blob/AI/Proiect_X_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import random
from itertools import product

class X_si_O():
    def __init__(self,jucator = 'X',tipul ='recompensa'): #self este folosit pentru a face referire la atributele clasei
        self.tabla = np.array(['__']*9).reshape(3,3)
        self.tipul = tipul
        self.combinatie_castigatoare = None # Tine evidenta mutarilor castigatoare facute de jucator
        self.prima_mutare = None # Tine evidenta primei mutari facute de jucator
        # Initializarea atributelor fiecarui jucator (X sau O)
        if jucator == 'X':
            self.eu ='X'
            self.id = 1
            self.adversar = 'O'
        else:
            self.eu = 'O'
            self.id = 2
            self.adversar = 'X'

        self.joc_incheiat = False # indica daca s-a terminat jocul
        # Mapare a reprezentarii actiunii in tabla la reprezentarea actiunii in tuplu
        self.b_catre_s = {'__':0,'X':1,'O':2}
        # Maparea inversa
        self.s_catre_b = {0:'__',1:'X',2:'O'}

        # Crearea maparii de la pozitii 2D in tabla la pozitii 1D in tuplu, pozitii = casutele libere pe tabla.
        # Perechile sunt apoi adaugate în dictionar, astfel încat fiecare pozitie 2D este asociata cu un indice unic în 1D.
        pozitii = self.pozitii_libere()
        self.b2_catre_s1 = {pozitie:i for (i,pozitie) in enumerate(pozitii)}

        #Crearea maparii invers, din 1D in 2D
        self.s1_catre_b2 = {i:pozitie for (i,pozitie) in enumerate(pozitii)}

        # Starea curenta a jocului
        self.stare_initiala = self.stare_tabla()

        # Initializarea tuturor starilor posibile ale jocului
        l_o_l = [list(range(3)) for _ in range(9)] #toate combinatiile posibile de valori pentru cele 9 casute ale tablei de joc
        stari = set(product(*l_o_l)) #set care contine toate starile posibile ale jocului

        #Starile jucatorului X includ starile cu numar impar de spatii libere si ambii jucatori au ocupat un numar egal de pozitii
        #Starile jucatorului O vin dupa jucatorul X, asa ca starile jucatorului O includ starile cu numar par de spatii libere si
        # unde jucatorul X a ocupat cu o pozitie mai mult decat jucatorul O
        jucatorX_stari = {stare for stare in stari if (stare.count(0)%2 == 1 and stare.count(1)==stare.count(2))}
        jucatorO_stari =  {stare for stare in stari if (stare.count(0)%2 == 0 and stare.count(1)==(stare.count(2)+1))}

        #Stari
        if jucator == 'X':
            self.stari_eu = jucatorX_stari
        else:
            self.stari_eu = jucatorO_stari


    def reseteaza_tabla(self):
        #Functie pentru resetarea jocului si a tablei la starea initiala
        self.tabla = np.array(['__']*9).reshape(3,3)
        self.stare_initiala = self.stare_tabla()
        self.joc_incheiat = False
        self.combinatie_castigatoare = None
        self.prima_mutare = None

    def arata_tabla(self):
        #Tabla e sub forma de data frame
        return pd.DataFrame(self.tabla)

    def stare_tabla(self):
        #Converteste tabla intr-un tuplu pentru usurinta implementarii
        return tuple([self.b_catre_s[x] for x in np.ravel(self.tabla)])

    @staticmethod #Metoda statica ne permite accesul la clasa fara a crea o instanta a acesteia
    def mutari_posibile(stare):
        # returneaza o lista cu indicii casutelor libere din stare
        return [i for i,x  in enumerate(stare) if x ==0]

    def este_incheiat(self):
        if not np.any(self.tabla == '__') :
            self.joc_incheiat = True
        return self.joc_incheiat

    def pozitii_libere(self):
        x,y = np.where(self.tabla =='__')
        return[(x,y) for x,y in zip(x,y)] #returneaza o lista, unde fiecare tuplu reprezinta coordonatele (x, y) libere din tabla


    def victorie(self,jucator):
        if np.all(self.tabla[0,:] == jucator):
            self.combinatie_castigatoare  = 'R1'
        elif np.all(self.tabla[1,:] == jucator):
            self.combinatie_castigatoare  = 'R2'
        elif np.all(self.tabla[2,:] == jucator):
            self.combinatie_castigatoare  = 'R3'
        elif np.all(self.tabla[:,0] == jucator):
            self.combinatie_castigatoare  = 'C1'
        elif np.all(self.tabla[:,1] == jucator):
            self.combinatie_castigatoare  = 'C2'
        elif np.all(self.tabla[:,2] == jucator):
            self.combinatie_castigatoare  = 'C3'
        elif np.all(self.tabla.diagonal()== jucator):
            self.combinatie_castigatoare  = 'D1'
        elif  np.all(np.fliplr(self.tabla).diagonal()== jucator):
            self.combinatie_castigatoare  = 'D2'
        else:
            return False
    #Daca nu se gaseste nicio combinatie castigatoare, returneaza False.
    #In caz contrar, returneaza True pentru a indica ca jucatorul a castigat.
        return True

    def mutarea_mea(self,pozitie):
        #Completeaza tabla pe pozitia data cu actiunea jucatorului
        assert pozitie[0] >= 0 and pozitie[0] <= 2 and pozitie[1] >= 0 and pozitie[1] <= 2 , "pozitie incorecta"
        assert self.tabla[pozitie] == "__" , "pozitie deja ocupata"
        assert np.any(self.tabla == '__') , "tabla este completa"
        assert self.victorie(self.eu) == False and self.victorie(self.adversar)== False , "jocul a fost deja castigat"
        self.tabla[pozitie] = self.eu
        eu_victorie = self.victorie(self.eu)
        adversar_victorie = self.victorie(self.adversar)

        if self.tipul == 'recompensa':
            if eu_victorie:
                self.joc_incheiat = True
                return 1

            elif adversar_victorie:
                self.joc_incheiat = True
                return -1

            else:
                return 0
        #Daca tipul este 'recompensa', se verifica daca jucatorul curent (eu) a castigat. Daca da, jocul este incheiat,se returneaza 1
        # Daca adversarul a castigat, jocul este incheiat si se returneaza -1
        elif self.tipul == 'penalizare':
            if eu_victorie:
                self.joc_incheiat = True
                return 0
            elif adversar_victorie:
                self.joc_incheiat = True
                return -10
            else:
                return -1

    def mutare_adversar(self,pozitie):
        #Completeaza tabla pe pozitia data cu actiunea adversarului
        assert pozitie[0] >= 0 and pozitie[0] <= 2 and pozitie[1] >= 0 and pozitie[1] <= 2 , "pozitie incorecta"
        assert self.tabla[pozitie] == "__" , "pozitie deja ocupata"
        assert np.any(self.tabla == '__') , "tabla este completa"
        assert self.victorie(self.eu) == False and self.victorie(self.adversar)== False , "jocul a fost deja castigat"
        self.tabla[pozitie] = self.adversar


    def alege_cea_mai_buna_mutare(self, Q, tip_mutare, eps=None):
        '''pentru a estima valoarea optima a unei actiuni intr-o anumita stare folosim Q. Functia Q atribuie valori pentru fiecare stare si actiune posibila
        '''
        # Se parcurge fiecare actiune posibila
        stare_curenta = self.stare_tabla()
        mutari =  self.mutari_posibile(stare_curenta)
        cea_mai_buna_mutare = []
        cea_mai_buna_mutare_valoare = -np.Inf #orice valoare va fi mai mare decat -inf, ea fiind valoarea de pornire

        for mutare in mutari:
            Q_s_a = Q[stare_curenta][mutare] #Se obtine valoarea Q pentru actiunea curenta in functie de starea curenta
            if Q_s_a == cea_mai_buna_mutare_valoare:
                cea_mai_buna_mutare.append(mutare)
            elif Q_s_a > cea_mai_buna_mutare_valoare:
                cea_mai_buna_mutare = [mutare]
                cea_mai_buna_mutare_valoare = Q_s_a
        cea_mai_buna_mutare = random.alegere(cea_mai_buna_mutare)
        #daca tipul mutarii este greedy, algoritmul alege sa urmeze cea mai buna estimare a actiunii pentru a maximiza recompensa, iar cu cat eps e mai mare cu atat se va explora mai mult
        if tip_mutare == 'greedy':
            return self.s1_catre_b2[cea_mai_buna_mutare]
        else:
            assert eps != None , "include eps"
            nr_mutari = len(mutari) # numarul de mutari
            p = np.full(nr_mutari,eps/nr_mutari) #probabilitati egal distribuite pentru a ne asigura ca fiecare actiune are sansa de a fi aleasa
            # indexul celei mai bune mutari
            cea_mai_buna_mutare_i = mutari.index(cea_mai_buna_mutare)
            p[cea_mai_buna_mutare_i]+= 1 - eps # 1-eps asigura o probabilitate mai mare pentru alegerea actiunii considerate cea mai buna
            return self.s1_catre_b2[np.random.alegere(mutari,p=p)] # actiunile cu probabilitati mai mari vor fi alese mai des, dar exista totusi sansa de a alege actiuni cu probabilitati mai mici


In [ ]:
t_tabla_X = X_si_O(jucator = 'X',tipul ='recompensa')
t_tabla_X.arata_tabla()

,0,1,2
0,__,__,__
1,__,__,__
2,__,__,__


In [ ]:
t_tabla_X.mutarea_mea((0,0))
t_tabla_X.arata_tabla()

,0,1,2
0,X,__,__
1,__,__,__
2,__,__,__


In [ ]:
t_tabla_X.mutare_adversar((0,1))
t_tabla_X.arata_tabla()

,0,1,2
0,X,O,__
1,__,__,__
2,__,__,__


In [ ]:
t_tabla_X.mutarea_mea((1,0))
t_tabla_X.arata_tabla()


,0,1,2
0,X,O,__
1,X,__,__
2,__,__,__


In [ ]:
t_tabla_X.mutare_adversar((1,1))
t_tabla_X.arata_tabla()

,0,1,2
0,X,O,__
1,X,O,__
2,__,__,__


In [ ]:
t_tabla_X.mutarea_mea((2,0))
t_tabla_X.arata_tabla()

,0,1,2
0,X,O,__
1,X,O,__
2,X,__,__
